## Preditor de cliques
#### A ideia desse projeto é criar um algoritmo preditor de cliques. Preditores de cliques são importantes ferramentes em meio a publicidade, podendo se otimizar o impulsionamento de anuncios de acordo a perfis encontrados em meio a dados. 
#### Nesse modelo, será analisado um banco de dados https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/overview que nos propoe construir uma ferramenta que consiga prever possiveis cliques fraudulentos em anúnicos de aplicativos. 
#### Meu objetivo é criar um preditor de cliques estudandos os dados das bases de dados, onde ao final, usarei duas formas de predição e mostrarei a cobertura alcançada para prever se de fato os cliques que chegam ao seu objeitvo, o download do aplicativo que o anúncio se referia.

#### O link para o git com as bases de dados: https://github.com/WillianAUK/regression_prediction_clicks

In [2]:
#Importando bibliotecas utilizadas no preditor ou para vizualização

import pandas as pd
import numpy as np
import timeit
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import SGDClassifier

### Montando funções responsaveis pelo treinamento do modelo através de regressão logística usando Gradiente descendente 

In [3]:
#Função logistica
def sigmoid(input):
    return 1.0 / (1 + np.exp(-input))

In [4]:
# Definindo a função que calcula a previsão y(x) dado os pesos.
def compute_prediction(X, weights):
    """ Calcule a previsão y_hat com base nos pesos atuais
    Args:
        X (numpy.ndarray)
        weights (numpy.ndarray)
    Returns:
        numpy.ndarray, y_hat of X under weights """
    
    z = np.dot(X, weights)
    predictions = sigmoid(z)
    return predictions

In [5]:
# Definindo a função para continuar atualizando os pesos de acordo ao gardiente descendente. 
def update_weights_gd(X_train, y_train, weights, learning_rate):
    """ Atualizar pesos em um passo
    Args:
        X_train, y_train (numpy.ndarray, training data set)
        weights (numpy.ndarray)
        learning_rate (float)
    Returns:
        numpy.ndarray, updated weights"""
    
    predictions = compute_prediction(X_train, weights)
    weights_delta = np.dot(X_train.T, y_train - predictions)
    m = y_train.shape[0]
    weights += learning_rate / float(m) * weights_delta
    return weights

In [6]:
# Definindo a definindo a função de calcula do custo j(w)
def compute_cost(X, y, weights):
    """ 
    Calcule o custo J (w)
    Args:
        X, y (numpy.ndarray, data set)
        weights (numpy.ndarray)
    Returns:
        float 
    """
    predictions = compute_prediction(X, weights)
    cost = np.mean(-y * np.log(predictions) - (1 - y) * np.log(1 - predictions))
    return cost


In [7]:
# Função que conecta as funções na função de treinamento do modelo, atualizando o vetor de pesos em cada iteração e 
# imprimindo o custo atual para cada 100 (podem ser outros valores) iterações afim de garantir que o custo está diminuindo 
# e que as coisas estão no caminho certo.

def train_logistic_regression(X_train, y_train, max_iter, learning_rate, fit_intercept=False):
    """ 
    Treine um modelo de regressão linear com gradiente descendente
    Args:
        X_train, y_train (numpy.ndarray, conjunto de dados de treinamento)
        Max_iter (int, número de iterações)
        Learning_rate (float)
        Fit_intercept (bool, com uma interceptação w0 ou não)
    Retorna:
        Numpy.ndarray, pesos aprendidos
    """
    if fit_intercept:
        intercept = np.ones((X_train.shape[0], 1))
        X_train = np.hstack((intercept, X_train))
    weights = np.zeros(X_train.shape[1])
    for iteration in range(max_iter):
        weights = update_weights_gd(X_train, y_train, weights, learning_rate)
        # Check the cost for every 100 (for example) iterations
        if iteration % 100 == 0:
            print(compute_cost(X_train, y_train, weights))
    return weights
    

In [8]:
# Definindo função que preve os resultados de novas entradas usando o medelo treinado 
def predict(X, weights):
    if X.shape[1] == weights.shape[0] - 1:
        intercept = np.ones((X.shape[0], 1))
        X = np.hstack((intercept, X))
    return compute_prediction(X, weights)

###  Nesse passo, importo as bases de dados que serão utilizadas para treinar o modelo (df_train = train.csv) e a base para testar o modelo treinado(df_teste = test.csv)

In [9]:
# Aqui o n_rows vai definir um limite de linhas a serem importadas das bases de dado, pois devido ao 
# tamanho dos arquivos, a memória do computador se tornou limitada. Sendo assim defini importar 1000000 
# de linhas, para não sobrecarregar minha maquina.

n_rows = 1000000  
df_train = pd.read_csv('train.csv', nrows = n_rows)
df_teste = pd.read_csv('test.csv', nrows = n_rows)

In [10]:
# Demonstrando a base de dados de treino que apresenta os seguintes atributos:
# Ip: endereço ip do clique.
# App: id do app para marketing.
# Device: ID do tipo de dispositivo do telefone celular do usuário (por exemplo, iphone 6 plus, iphone 7, huawei mate 7 etc.)
# Os: id da versão do sistema operacional do telefone celular do usuário
# Channel: id do canal do editor de anúncios para celular
# Click_time: data / hora do clique (UTC)
# Attribute_time: se o usuário baixar o aplicativo depois de clicar em um anúncio, é o momento do download do aplicativo
# Is_attributed: o destino que deve ser previsto, indicando que o aplicativo foi baixado

df_train

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,83230,3,1,13,379,2017-11-06 14:32:21,NaN,0
1,17357,3,1,19,379,2017-11-06 14:33:34,NaN,0
2,35810,3,1,13,379,2017-11-06 14:34:12,NaN,0
3,45745,14,1,13,478,2017-11-06 14:34:52,NaN,0
4,161007,3,1,13,379,2017-11-06 14:35:08,NaN,0
...,...,...,...,...,...,...,...,...
999995,29748,9,1,12,134,2017-11-06 16:21:51,NaN,0
999996,124520,12,1,15,178,2017-11-06 16:21:51,NaN,0
999997,206446,18,1,42,107,2017-11-06 16:21:51,NaN,0
999998,167577,12,1,13,265,2017-11-06 16:21:51,NaN,0


In [11]:
# A base de teste é é um pouco diferente, os atributos 'atribute_time' e 'is_atribute' não estão presentes já que esse é o
# objetivo do treinamento e é acrecentado o atributo 'click_id' que serve de referência.

df_teste

,click_id,ip,app,device,os,channel,click_time
0,0,5744,9,1,3,107,2017-11-10 04:00:00
1,1,119901,9,1,3,466,2017-11-10 04:00:00
2,2,72287,21,1,19,128,2017-11-10 04:00:00
3,3,78477,15,1,13,111,2017-11-10 04:00:00
4,4,123080,12,1,13,328,2017-11-10 04:00:00
...,...,...,...,...,...,...,...
999995,999995,53436,18,1,23,376,2017-11-10 04:17:35
999996,999996,25714,6,1,13,125,2017-11-10 04:17:35
999997,999997,31139,13,1,18,477,2017-11-10 04:17:35
999998,999998,114636,18,1,78,439,2017-11-10 04:17:35


### Filtrando os dados de Treino  

In [12]:
# Agora com os dados identificados,é preciso filtrar os que serão utlizados no treinamento.
# Para o elemento X do treino, das 8 colunas disponiveis do df_train, serão utlizadas a colula 'app' e 'channel'
# que serão correlacionadas com o elemento Y contendo 'is_tributed' que assume valor 0 e 1, sendo 1 quando o clicque 
# é completado com um download. 
# Já n_train é o número de amostras que vamos utlizar da nossa base de dados para treinar o modelo.

X = df_train.drop(columns=['ip','device', 'os', 'click_time', 'attributed_time', 'is_attributed']).values
Y = df_train['is_attributed'].values
n_train = 100000

### Iniciando o treinamento do modelo 

In [13]:
# Aqui treinaremos o modelo com 100000 iterações numa taxa de aprendizado de 0,01.
# Utilizei a Função timeit para demonstrar o tempo que se leva ao realizar esse treinamento usando a 
# a tecnica de regressão usando o gradiente descendente. As saídas demonstram que o modelo vai ficando
# mais preciso com o passar das iterações.

start_time = timeit.default_timer()
weights = train_logistic_regression(X, Y, max_iter=100000, learning_rate=0.01, fit_intercept=True)

0.43154985790067246
0.3025826107204381
0.17737017017550225
0.05692279128509142
0.022405690925133382
0.022128963129672254
0.02203863084552145
0.0219976261552673
0.021972374027567132
0.02195256196806824
0.021934716381104422
0.02191761045619345
0.021900799230603594
0.02188411878609412
0.02186750816190703
0.021850944539260512
0.021834419322939263
0.021817929241935766
0.02180147302226457
0.021785050139667386
0.021768660351395373
0.021752303520372288
0.021735979549179517
0.021719688355296946
0.02170342986183449
0.021687203994073232
0.021671010678181453
0.021654849840742042
0.021638721408581224
0.02162262530870809
0.021606561468294493
0.021590529814669114
0.021574530275316316
0.021558562777876704
0.021542627250147633
0.02152672362008423
0.021510851815799738
0.021495011765566393
0.0214792033978158
0.02146342664113938
0.021447681424289048
0.021431967676177305
0.02141628532587784
0.02140063430262584
0.021385014535818446
0.02136942595501493
0.02135386848993726
0.021338342070470244
0.0213228466266

0.017390840867910606
0.017383038166317588
0.01737525124634016
0.017367480093160203
0.017359724692041147
0.01735198502831947
0.017344261087396232
0.017336552854728595
0.017328860315821003
0.017321183456216513
0.017313522261487895
0.01730587671722868
0.017298246809044293
0.01729063252254291
0.017283033843326624
0.017275450756982245
0.017267883249072427
0.01726033130512659
0.017252794910632075
0.01724527405102522
0.01723776871168251
0.017230278877911977
0.017222804534944403
0.01721534566792495
0.0172079022619048
0.017200474301832764
0.017193061772547296
0.017185664658768602
0.017178282945090842
0.017170916615974592
0.017163565655739423
0.01715623004855687
0.01714890977844336
0.017141604829253598
0.017134315184674058
0.017127040828216738
0.017119781743213144
0.0171125379128086
0.01710530931995673
0.01709809594741421
0.01709089777773588
0.01708371479326996
0.017076546976153687
0.017069394308309123
0.017062256771439334
0.017055134347024728
0.0170480270163197
0.01704093476034964
0.01703385755

0.015284953593523935
0.015281426673675564
0.01527790579633639
0.01527439094549791
0.015270882105206766
0.015267379259564333
0.015263882392726746
0.015260391488904545
0.015256906532362538
0.015253427507419596
0.015249954398448432
0.015246487189875419
0.015243025866180392
0.015239570411896441
0.01523612081160974
0.015232677049959335
0.015229239111636924
0.015225806981386706
0.015222380644005109
0.015218960084340722
0.015215545287293996
0.015212136237817081
0.015208732920913685
0.015205335321638723
0.015201943425098362
0.015198557216449617
0.015195176680900304
0.015191801803708759
0.015188432570183664
0.015185068965683965
0.01518171097561851
0.015178358585445976
0.015175011780674643
0.015171670546862253
0.015168334869615788
0.015165004734591212
0.015161680127493452
0.015158361034076064
0.015155047440141149
0.015151739331539096
0.015148436694168443
0.015145139513975689
0.01514184777695508
0.015138561469148515
0.015135280576645245
0.01513200508558178
0.015128734982141689
0.01512547025255543

In [14]:
# Tempo levado para treinar o modelo baseado em regressão gradiente descendente.
print ("Tempo gasto para realizar o treinamento do modelo: % 0.2f segundos."% (timeit.default_timer () -start_time))

Tempo gasto para realizar o treinamento do modelo:  4436.81 segundos.


### Filtrando os dados de teste do modelo treinado

In [15]:
# Aqui usando df_teste selecionaremos X_test contendo as colunas 'app' e 'channel' as mesma utilizadas 
# em X da base df_train.

X_test = df_teste.drop(columns=['click_id','ip', 'device', 'os', 'click_time']).values
Y_test = df_train['is_attributed'].values
X_test

array([[  9, 107],
       [  9, 466],
       [ 21, 128],
       ...,
       [ 13, 477],
       [ 18, 439],
       [  2, 469]], dtype=int64)

### Realizando a predição dos dados teste com o modelo treinado 

In [16]:
predictions = predict(X_test, weights)

In [17]:
# Obtendo e mostrando a cobertura de precisão que o modelo tilizando regressão por gradiente descendente alcançou 
# com os dados de 100000 amostras:
print('Amostras de treinamento: {0}, Cobertura de precisação alcançada:{1:.3f}'.format(n_train, roc_auc_score(Y_test, predictions)))
cobertura_RGD = roc_auc_score(Y_test, predictions)

Amostras de treinamento: 100000, Cobertura de precisação alcançada:0.501


## Agora farei o treinamento me basendo na abordagem de regressão logística usando  gradiente estocástico
#### Porém aqui utlizarei o módulo SDGClassifier do scikit-learn

In [21]:
# Definindo os parametros do SDG

sgd_lr = SGDClassifier(loss='log', penalty=None, fit_intercept=True, learning_rate='constant', eta0=0.01)

### Treinando o modelo SDG 

In [22]:
# Treinando o modelo SDG e marcando tempo para tal

start_time = timeit.default_timer()
sgd_lr.fit(X, Y)

SGDClassifier(eta0=0.01, learning_rate='constant', loss='log', penalty=None)

In [23]:
print ("Tempo gasto para realizar o treinamento do modelo com SDG: % 0.2f segundos."% (timeit.default_timer () -start_time))

Tempo gasto para realizar o treinamento do modelo com SDG:  6.33 segundos.


### Realizando a predição do modelo treinado com SDG

In [24]:
pred = sgd_lr.predict_proba(X_test)[:, 1]

In [25]:
# Obtendo e mostrando a cobertura e precisão do modelo treinado em SDG.

print('Amostras de treinamento: {0}, Cobertura de precisação alcançada:{1:.3f}'.format(n_train, roc_auc_score(Y_test, pred)))
cobertura_SDG = roc_auc_score(Y_test, pred)

Amostras de treinamento: 100000, Cobertura de precisação alcançada:0.500


## Conclusões 
#### RGD > tempo de treino: 4436.81 segundos e cobertura: 0.501
#### SDG > tempo de treino: 6.33 segundos e cobertura: 0.500

####      Utilizando o modelo de treinamento baseado em regressão logística de gradiente descendente foi possivel alcançar uma cobertura de 0.501. O resultado obtido poderá não ser muito útil dada a exigencia de cobertura. Outro problema dessa abordagem é o tempo de execução para treinar o modelo, pois apesar de não ser usado toda  a base de dados e treinar com 100000 interações a uma taxa de 0,01 custou um tempo muito grande. Se o conjunto de dados fosse maior, o que é muito possível de acontecer, o tempo para treinar o modelo aumenteria exponencialmente.

#### O segundo exemplo, o modelo de treinamento de regressão logística usando descida do gradiente estocástico usando o módulo SDGClassifier do scikit-learn. Nessa abordagem, obtivemos uma grande melhora no quesito tempo de aprendizado do modelo e foi obtido uma cobertura semelhante de precisão com o modelo.